In [ ]:
"""
This notebook is a scratch book about downloading the dew (Date Estimation in the Wild) dataset.

The final script downloading the dataset is "download_dew.py".
"""

In [3]:
import pandas as pd
import requests
import time
import os

In [6]:
df = pd.read_csv('data/dew/meta.csv')

In [7]:
until = 1950
# download_folder = 'data/dew/images_until_1950'
download_folder = '/volumes/512GB 5200U TM/datasets/dew/until_1950/'

In [8]:
df_selected = df[df['GT'] <= until]

In [9]:
df_selected.head(10)

,img_id,GT,date_taken,date_granularity,url,username,title,license,license_url
869,8796545829,1930,1930-01-01 10:47:35,0,https://farm8.staticflickr.com/7281/8796545829...,Dawn 1972,1930,All Rights Reserved,NaN
870,8404012017,1930,1930-01-01 00:00:00,6,https://farm9.staticflickr.com/8336/8404012017...,Ecomuseo Valsugana | Croxarie,"Strigno, 1930?",All Rights Reserved,NaN
871,11668255363,1930,1930-01-01 00:00:00,8,https://farm4.staticflickr.com/3707/1166825536...,FotoGuy 49057,Yokohama Japan 1930,All Rights Reserved,NaN
872,25578292114,1930,1930-01-01 00:00:00,6,https://farm2.staticflickr.com/1689/2557829211...,mama knipst!,Düsseldorf 1930 Grafenwerth - Foto aus einem a...,All Rights Reserved,NaN
873,2402788766,1930,1930-01-01 00:00:00,6,https://farm4.staticflickr.com/3115/2402788766...,Striderv,Dorothy Steel in Montreal,All Rights Reserved,NaN
874,26159050312,1930,1930-01-01 00:00:00,6,https://farm2.staticflickr.com/1653/2615905031...,mama knipst!,Kirschblüte Metternich 1930 - Foto aus einem a...,All Rights Reserved,NaN
875,3151609787,1930,1930-01-01 00:00:00,6,https://farm4.staticflickr.com/3122/3151609787...,Infrogmation,Summer 1930,Attribution License,https://creativecommons.org/licenses/by/2.0/
876,3463533474,1930,1930-01-01 00:00:00,8,https://farm4.staticflickr.com/3559/3463533474...,phlubdr,interior 1930,Attribution-NonCommercial License,https://creativecommons.org/licenses/by-nc/2.0/
877,9797614024,1930,1930-01-01 00:00:00,8,https://farm4.staticflickr.com/3766/9797614024...,Fotos colomines,"Santa Coloma de Queralt, Escoles Nacionals Gru...",All Rights Reserved,NaN
878,5442930268,1930,1930-01-01 00:00:00,6,https://farm6.staticflickr.com/5138/5442930268...,archivesplus,"Daily Herald Newspaper Building (interior), Ma...",All Rights Reserved,NaN


In [24]:
downloaded_images = os.listdir(download_folder)

downloaded_ids = []
for image_path in downloaded_images:

    file_name = image_path.split('.')[0]

    if file_name != '':
        img_id = int(file_name.split('_')[0])  # first part before "_" is image id in dataset
        downloaded_ids.append(img_id)

# image_ids = [image_path.split('.')[0] for image_path in already_processed_images]
# image_ids = [int(entry.split('_')[0]) for entry in image_ids if entry != '']

# filter out bad responses from last attempt
with open('bad_responses.txt') as f:
    bad_responses = [line.rstrip('\n') for line in f]

print('before remove', len(df_selected))

remaining_images = df_selected[~df_selected['url'].isin(bad_responses)]
print('after removing bad responses', len(remaining_images))

# all_image_ids = df_selected['img_id']
# print('before remove', len(all_image_ids))

# filter out already downloaded images
remaining_image_ids = remaining_images[~remaining_images['img_id'].isin(downloaded_ids)]['img_id']
print('after removing downloaded', len(remaining_image_ids), '\n')


before remove 98227
after removing bad responses 98217
after removing downloaded 57894 



In [22]:
succesful_downloads = []
errors = []
bad_responses = []

total_images = len(remaining_image_ids)
count = 0
total_time = 0

for img_id in remaining_image_ids:
# for _, row in df_selected.iterrows():
    start_time = time.time()
    count += 1

    row = df_selected[df_selected['img_id'] == img_id].iloc[0]

    url = row['url']
    print(f'Downloading no.{count}:', url)

    file_name = str.split(url, '/')[-1]
    try:
        response = requests.get(url)
        print(response.status_code, response.headers['content-type'])

        if response.headers['content-type'] == 'image/jpeg':
            f = open(f'{download_folder}/{file_name}','wb')
            f.write(response.content)
            f.close()

            succesful_downloads.append(url)
            print('download successful')
        else:
            bad_responses.append(url)

            with open('bad_responses.txt', 'a') as f:
                f.write(url)
                f.write('\n')

            print('bad response')

    except IOError:
        errors.append(url)
        print('failed downloading:', url)

    end_time = time.time()
    took_time = end_time - start_time
    total_time += took_time
    left_images = total_images - count
    avg_time_per_image = total_time / count

    print()
    print('took:', round(took_time, 1), 'secs')
    print('time total:', round(total_time, 1), 'secs')
    print('avg time:', round(avg_time_per_image, 1), 'secs')
    print('left:', left_images, 'est:', round(avg_time_per_image * left_images / 60, 0), 'mins', '\n')

    # print(i, url, file_name)
    # if len(succesful_downloads) == 50:
    if count == 10:
        break

print(f'Downloaded {len(succesful_downloads)} images, {len(errors)} errors, {len(bad_responses)} bad responses.')

410 text/html;charset=utf-8
bad response

took: 0.1 secs
time total: 0.1 secs
avg time: 0.1 secs
left: 57903 est: 91.0 mins 

404 text/html; charset=utf-8
bad response

took: 0.5 secs
time total: 0.6 secs
avg time: 0.3 secs
left: 57902 est: 281.0 mins 

404 text/html; charset=utf-8
bad response

took: 0.5 secs
time total: 1.0 secs
avg time: 0.3 secs
left: 57901 est: 334.0 mins 

404 text/html; charset=utf-8
bad response

took: 0.2 secs
time total: 1.2 secs
avg time: 0.3 secs
left: 57900 est: 293.0 mins 

404 text/html; charset=utf-8
bad response

took: 0.4 secs
time total: 1.7 secs
avg time: 0.3 secs
left: 57899 est: 319.0 mins 

404 text/html; charset=utf-8
bad response

took: 0.2 secs
time total: 1.8 secs
avg time: 0.3 secs
left: 57898 est: 293.0 mins 

404 text/html; charset=utf-8
bad response

took: 0.2 secs
time total: 2.0 secs
avg time: 0.3 secs
left: 57897 est: 274.0 mins 

404 text/html; charset=utf-8
bad response

took: 0.2 secs
time total: 2.2 secs
avg time: 0.3 secs
left: 57